<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">2014 South Napa Earthquake</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Multiple Waveforms Request</div>
        </div>
    </div>
</div>

### **2017 CIG-LLNL Computational Seismology Workshop**


##### Authors:
* Artie Rogers
* Lion Krischer ([@krischer](https://github.com/krischer))
---

3) Request waveforms for a single evnet, but from multiple network-channel combinations.

If you ran 2) single net-chan request (previous notebook), then you will need to remove event directory "72282711".

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

In [36]:
from os import path, makedirs
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

In [37]:
# Set eventid here
event_id = 72282711

# Continuous data generally available from these networks
netchan_list = ['BK.BH', 'NC.HN', 'NP.HN' ]
minimum_length = .95

# Note network 'CE' only provides short waveform segments, sometimes only 55 seconds of data)
#netchan_list = [ 'CE.HN' ]
#minimum_length = 0.2

maxradius = 0.5
tbefore = 20.
tafter = 240.
data_center = 'NCEDC'

Set client, create event directory (if it does not already exist).
Query client to get event information into catalog.
Set domain restrictions for waveform query.

In [38]:
client = Client(data_center)

# Create directory for this event, if it does not exist
event_dir = str(event_id)
if path.exists(event_dir)==False:
    print('make directory '+event_dir)
    makedirs(event_dir)
    raw_wf_data_dir = event_dir+'/RAW/'
    makedirs(raw_wf_data_dir)
    sta_metadata_dir = event_dir+'/_station_xml/'
    makedirs(sta_metadata_dir)
else:
    print('directory exists')
            
# Query client for this particular event_id, returns catalog
catalog = client.get_events(eventid=event_id)
format='QUAKEML'
catalog.write(event_dir+'/'+str(event_id)+'.'+format, format=format)
print ('... catalogue saved to '+str(event_id)+'.'+format)
# Create event origin from the 0-th origin in the catalog
ev = catalog[0]
origin_time = ev.origins[0]['time']
evlon = ev.origins[0]['longitude']
evlat = ev.origins[0]['latitude']
print('origin_time: ', origin_time, 'lat, lon: ', str(evlat), str(evlon) )

# Domain restriction for waveform query
domain = CircularDomain(latitude=evlat, longitude=evlon,
                            minradius=0, maxradius=maxradius)


make directory 72282711
... catalogue saved to 72282711.QUAKEML
origin_time:  2014-08-24T10:20:44.070000Z lat, lon:  38.2151667 -122.3123333


Loop over netchan (network-channel combinations).
Run waveform query to return waveforms.

In [39]:
for netchan in netchan_list:
    net = netchan.split('.')[0]
    chan = netchan.split('.')[1]
    channels = [chan+'[ZNE]']
    print ('netchan: ', netchan, ' net: ', net, ' chan: ', chan, ' channels: ', channels)
    
    # Restrictions for waveform query, this time use minimum length restriction
    restrictions = Restrictions(starttime=origin_time - tbefore,
            endtime=origin_time + tafter,
            network=net,
            reject_channels_with_gaps=False,
            minimum_length=minimum_length,
            channel_priorities=channels)
    
    # Use massdownloader
    mdl = MassDownloader(providers=[data_center])
    
    try:
        mdl.download(domain, restrictions, mseed_storage=raw_wf_data_dir,
                     stationxml_storage=sta_metadata_dir)
    except:
        print ('\n\n#########     DOWNLOAD ERROR: moving on      ##########\n\n')

[2017-06-04 23:05:24,034] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for NCEDC.
[2017-06-04 23:05:24,038] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): NCEDC.
[2017-06-04 23:05:24,038] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2017-06-04 23:05:24,039] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Requesting unreliable availability.


netchan:  BK.BH  net:  BK  chan:  BH  channels:  ['BH[ZNE]']


[2017-06-04 23:05:25,814] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully requested availability (1.77 seconds)
[2017-06-04 23:05:25,856] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Found 8 stations (24 channels).
[2017-06-04 23:05:25,858] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Will attempt to download data from 8 stations.
[2017-06-04 23:05:25,860] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Status for 24 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2017-06-04 23:05:38,636] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 24 channels (of 24)
[2017-06-04 23:05:38,637] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Launching basic QC checks...
[2017-06-04 23:05:38,659] - obspy.clients.fdsn.mass_downloader - INFO: File '72282711/RAW/BK.CVS.00.BHE__20140824T102024Z__20140824T102444Z.mseed' has only 197.95 seconds of data. 247.00 are re

netchan:  NC.HN  net:  NC  chan:  HN  channels:  ['HN[ZNE]']


[2017-06-04 23:05:59,297] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully requested availability (1.72 seconds)
[2017-06-04 23:05:59,354] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Found 29 stations (87 channels).
[2017-06-04 23:05:59,356] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Will attempt to download data from 29 stations.
[2017-06-04 23:05:59,360] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Status for 87 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2017-06-04 23:06:05,766] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 37 channels (of 37)
[2017-06-04 23:06:08,564] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 50 channels (of 50)
[2017-06-04 23:06:08,565] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Launching basic QC checks...
[2017-06-04 23:06:08,680] - obspy.clients.fdsn.mass_downloa

netchan:  NP.HN  net:  NP  chan:  HN  channels:  ['HN[ZNE]']


[2017-06-04 23:07:30,754] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully requested availability (6.07 seconds)
[2017-06-04 23:07:30,800] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Found 70 stations (192 channels).
[2017-06-04 23:07:30,803] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Will attempt to download data from 70 stations.
[2017-06-04 23:07:30,811] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Status for 192 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2017-06-04 23:07:35,707] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 18 channels (of 50)
[2017-06-04 23:07:36,108] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 27 channels (of 50)
[2017-06-04 23:07:36,654] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NCEDC' - Successfully downloaded 38 channels (of 50)
[2017-06-04 23:07:40,005] - obspy.clients.

Check in event directory "72282711". There should be:
    a quakeml file ("72282711.xml"); 
    a "_station_xml" direcotry with Station.XML files, named {NET}.{CHAN}.xml; and 
    a "RAW" direcotry with mseed files.